<a href="https://colab.research.google.com/github/univer-coder/Deep-learning-course/blob/main/lecture02_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第2回講義 宿題

## 課題
今回のLessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) をソフトマックス回帰によって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- 訓練データは`x_train`， `y_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください．**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第2回 機械学習基礎」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第2回 機械学習基礎 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### データの読み込み（このセルは修正しないでください）

In [5]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/x_train.npy')
    y_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/y_train.npy')

    # テストデータ
    x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255

    return x_train, y_train, x_test

### ソフトマックス回帰の実装

In [6]:
x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

def softmax(x):
    x -= x.max()
    x_exp =  np.exp(x)
    return  x_exp / x_exp.sum()

# 重み
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')  # 重み: (784, 10)
b = np.zeros(shape=(10,)).astype('float32')

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, t, eps=1.0):
    global W, b

    batch_size = x.shape[0]

    # 予測
    y_hat =  softmax(np.matmul(x, W) + b)

    # 目的関数の評価
    cost = (- t * np_log(y_hat)).sum(axis=1).mean()
    delta = y_hat - t  # (batch_size, 出力の次元数)

    # パラメータの更新
    dW = np.matmul(x.T, delta) / batch_size  # (入力の次元数, 出力の次元数)
    db = np.matmul(np.ones(shape=(batch_size,)), delta) / batch_size  # (出力の次元数,)
    W -= eps * dW
    b -= eps * db

    return cost

def valid(x, t):
    y_hat = softmax(np.matmul(x, W) + b)
    cost = (- t * np_log(y_hat)).sum().mean()

    return cost, y_hat

for epoch in range(1):
    cost = train(x_train, y_train)

    cost, y_pred = valid(x_valid, y_valid)

submission = pd.Series(y_pred, name='label')
submission.to_csv('drive/MyDrive/松尾研AI講座/DL基礎講座/第2回課題/submission_pred.csv', header=True, index_label='id')

ValueError: Data must be 1-dimensional, got ndarray of shape (6000, 10) instead